# GAIA PSF Simulation

First attempt to simulate close binary stars observation using the gaia telescope. The latter is imagined as a rectangular pupil, with no pixel binning or aberrations

#### Initialization

In [10]:
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [26]:
%matplotlib qt

QSocketNotifier: Can only be used with threads started with QThread


In [18]:
import poppy
import numpy as np
import astropy.units as u
from astropy.table import QTable
from astropy.io import fits
from matplotlib import pyplot as plt
from astropy import convolution as c
from simulator import GaiaTelescopeV0
from opticalib import load_fits

Gpb = QTable.read("data/gaiaDR3passband.fits")
weights = Gpb.filled(0)

In [ ]:
poppy.conf.use_multiprocessing = False
poppy.conf.n_processes = 20
poppy.conf.double_precision = False

M1 = poppy.RectangleAperture(name="Primary Mirror", width=1.45*u.m, height=0.5*u.m)
# M2 = poppy.RectangleAperture(name="Beam Collimator / Secondary Mirror",width=0.35*u.m, height=0.2*u.m)

In [31]:
telescope = poppy.OpticalSystem(name="Gaia", oversample=1000, verbose=True)

telescope.add_pupil(M1)
# telescope.add_pupil(M2)

# oversampled sky
telescope.add_detector(name="AF CCD", pixelscale=0.001*1000*(u.arcsec/u.pixel), fov_arcsec=4000*u.arcsec)

psf = telescope.calc_psf(progressbar=True, return_intermediates=True, return_final=True, source={'wavelengths': Gpb['lambda'], 'weights': weights['G']})

2025-09-17 14:41:30,523 - INFO - Initialized OpticalSystem: Gaia
2025-09-17 14:41:30,525 - INFO - Added pupil plane: Primary Mirror
2025-09-17 14:41:30,542 - INFO - Added detector with pixelscale=1.0 arcsec / pix and oversampling=1000: AF CCD
2025-09-17 14:41:30,544 - INFO - Calculating PSF with 781 wavelengths
2025-09-17 14:41:30,545 - INFO - User requested saving intermediate wavefronts in call to poppy.calc_psf
  0%|                                                   | 0/781 [00:00<?, ?it/s]2025-09-17 14:41:30,546 - INFO -  Propagating wavelength = 320 nm
2025-09-17 14:41:30,547 - INFO - You did not define npix either on the OpticalSystem or its first optic; defaulting to 1024 pixels.
  0%|                                                   | 0/781 [00:00<?, ?it/s]


OutOfMemoryError: Out of memory allocating 16,384,000,000 bytes (allocated so far: 578,327,552 bytes).

In [30]:
plt.imshow(psf[0][0].data, origin='lower', cmap='inferno')

In [ ]:
ccd = poppy.OpticalSystem(name="CCD", oversample=4, verbose=True)
ccd.add_pupil(M1)
ccd.add_detector(name="CCD", pixelscale=0.059*4*(u.arcsec/u.pixel), fov_arcsec=6*u.arcsec)

psf_ccd = ccd.calc_psf(progressbar=True, return_final=True, source={'wavelengths': Gpb['lambda'], 'weights': weights['G']})
ccd_psf = psf_ccd[0][0].data
ccd_psf = poppy.utils.rebin_array(ccd_psf, (1,3))

plt.figure(figsize=(8,8))
plt.imshow(ccd_psf, origin='lower', cmap='inferno')
plt.colorbar(shrink=0.8, label='Intensity')
plt.title("CCD PSF")
plt.xlabel("Pixels")
plt.ylabel("Pixels")
plt.show()

In [23]:
psf_1 = load_fits('data/simulations/PSFs/0.25mas_res/G_psf.fits')

In [27]:
plt.figure(figsize=(10,10))
plt.imshow(poppy.utils.rebin_array(psf_1, (59,177)), origin='lower', cmap='inferno')

In [28]:
plt.figure(figsize=(10,10))
plt.imshow(psf_1, origin='lower', cmap='inferno')